# Toronto neighborhoods' data scraping and wrangling

#### First, we import necessary libraries for scraping and wrangling

In [1]:
!pip3 install geocoder
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import numpy as np
import geocoder

You should consider upgrading via the 'pip install --upgrade pip' command.


#### Next, we scrape the table from the wikipedia page providing the list of postal codes in Canada

In [4]:
#get url using the requests library
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#parse html of the web page using html parser
soup = BeautifulSoup(source,'html.parser')

#open a csv file passing in the writing argument
csv_file = open('canada_data.csv','w')

#assign the column headers
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['PostalCode','Borough','Neighborhood'])

#parse table from the web page
table  = soup.find('table')

#iterate over rows to fill the rows of the csv file
rows = table.find_all('tr')
for row in rows:
    csv_row = []
    for cell in row.find_all('td'):
        csv_row.append(cell.get_text())
    csv_writer.writerow(csv_row)
print('data loaded to csv file')

data loaded to csv file


#### read the file into a pandas dataframe

In [5]:
toronto_df = pd.read_csv('canada_data.csv')
toronto_df = toronto_df.replace('\n','', regex=True)
toronto_df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


#### Then we remove rows without assigned Boroughs

In [6]:
#replace 'Not assigned' with nan
toronto_df.replace("Not assigned", np.nan, inplace = True)

#drop rows with nan values in Borough column
toronto_df.dropna(subset=["Borough"], axis=0, inplace=True)

#reset index of the rows
toronto_df.reset_index(drop=True, inplace=True)


toronto_df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,"Garden District, Ryerson"


#### Afterwards, we replace cells with 'Not assigned' neighborhood so the neighborhood will be the same as the borough.

In [7]:
toronto_df.replace("", "Borough", inplace = True)
toronto_df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [8]:
toronto_df.groupby('PostalCode')
toronto_df['Neighborhood'] = toronto_df['Neighborhood'].str.replace('/',',')
toronto_df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


#### Now we group neighborhoods with the same postal code

In [9]:
toronto_df.shape

(103, 3)